In [1]:
import pandas as pd
import numpy as np
import pickle

import statsmodels.api as sm
from statsmodels.formula.api import ols

from statsmodels.stats.multicomp import pairwise_tukeyhsd


# Perform statistical analysis on dataset.

In [2]:
with open('../data/cleaned_dataframe.pickle','rb') as read_file:
    df = pickle.load(read_file)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9551 entries, 4116 to 120626
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ID              9304 non-null   object        
 1   Facility        9551 non-null   object        
 2   testdate        9373 non-null   datetime64[ns]
 3   ageattest       9551 non-null   float64       
 4   Gender          9551 non-null   object        
 5   Country         9551 non-null   object        
 6   ethnicgroup     4517 non-null   object        
 7   height          9551 non-null   float64       
 8   weight          9551 non-null   float64       
 9   BMI             9551 non-null   float64       
 10  BetaMed         9286 non-null   float64       
 11  ANYCVD          9551 non-null   float64       
 12  COPD            8667 non-null   float64       
 13  Mode            9551 non-null   object        
 14  max_load_watts  755 non-null    object        
 15 

In [4]:
df.Facility.value_counts()

MAYO                           4937
Ball State                     3210
Hartford Hospital               254
Pennington                      247
MCH                             204
KUMC                            202
JHU                             157
University of Massachusetts     157
SCSU                             78
SAMMC                            72
SFSU                             21
U Montana                         9
GHCPS                             3
Name: Facility, dtype: int64

### Treadmill test analysis.

In [5]:
# Performing two-way ANOVA on TREADMILL tests to examine age and sex impact on RPE.

model_TM = ols('peak_rpe ~ C(age_group) + C(Gender) + C(age_group):C(Gender)',
            data=df[df.Mode=="TM"]).fit()
result_TM = sm.stats.anova_lm(model_TM, type=2)
  
# Print the result
print(result_TM)

                            df        sum_sq    mean_sq         F  \
C(age_group)               6.0     82.563298  13.760550  8.243699   
C(Gender)                  1.0      0.191692   0.191692  0.114839   
C(age_group):C(Gender)     6.0     12.865057   2.144176  1.284538   
Residual                8807.0  14700.822986   1.669220       NaN   

                              PR(>F)  
C(age_group)            6.389022e-09  
C(Gender)               7.347086e-01  
C(age_group):C(Gender)  2.604745e-01  
Residual                         NaN  


In [8]:
# Tukey post-hoc analysis for pair-wise comparisons.
# (only looking at within age based on results above).

tukey = pairwise_tukeyhsd(endog=df[df.Mode=="TM"]['peak_rpe'],
                          groups=df[df.Mode=="TM"]['age_group'],
                          alpha=0.05)
# print(tukey)

### Cycling test analysis.


In [10]:
# Performing two-way ANOVA on CYCLING tests to examine age and sex impact on RPE.

model_CY = ols('peak_rpe ~ C(age_group) + C(Gender) + C(age_group):C(Gender)',
            data=df[df.Mode=="CY"]).fit()
result_CY = sm.stats.anova_lm(model_CY, type=2)
  
# Print the result
print(result_CY)

                           df       sum_sq   mean_sq         F    PR(>F)
C(age_group)              6.0    43.641578  7.273596  4.684638  0.000109
C(Gender)                 1.0     8.708110  8.708110  5.608552  0.018138
C(age_group):C(Gender)    6.0     4.606137  0.767689  0.494439  0.812757
Residual                716.0  1111.696231  1.552648       NaN       NaN


In [13]:
# Tukey post-hoc analysis for pair-wise comparisons.
# (looking at age based on results above).

tukey = pairwise_tukeyhsd(endog=df[df.Mode=="CY"]['peak_rpe'],
                          groups=df[df.Mode=="CY"]['age_group'],
                          alpha=0.05)
# print(tukey)

In [15]:
# Performing three-way ANOVA (same as above but including test mode).

model = ols('peak_rpe ~ C(age_group) + C(Gender) + C(Mode)',
            data=df).fit()
result = sm.stats.anova_lm(model, type=2)
  
# Print the result
# print(result)